In [3]:
from path import HERE
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display
import sqlite3
import pandas as pd
import json

plt.style.use('ggplot')
sns.set(font_scale=1.2)

DB_PATH = HERE.joinpath("methods/optimization/optimizations.db")


def load_optimization_results(db_path):
    if not os.path.exists(db_path):
        print(f"Database file not found at {db_path}")
        return pd.DataFrame()

    conn = sqlite3.connect(db_path)

    # Query to get all optimization trials
    query = """
    SELECT 
        id, timestamp, study_name, trial_number, method, 
        parameters, metrics, runtime, extra_data, generation_arguments
    FROM 
        optimization_trials
    ORDER BY 
        timestamp DESC, study_name, trial_number
    """

    # Load the data
    df = pd.read_sql_query(query, conn)
    conn.close()

    # Parse JSON columns
    for col in ['parameters', 'metrics', 'extra_data', 'generation_arguments']:
        df[col] = df[col].apply(lambda x: json.loads(x) if x else {})

    # Extract key information from nested JSON
    df['SUR'] = df['metrics'].apply(lambda x: x.get('SUR', 0))
    df['DSN'] = df['metrics'].apply(lambda x: x.get('DSN', 0))
    df['TSN'] = df['metrics'].apply(lambda x: x.get('TSN', 0))

    # Extract dataset and data source from extra_data
    df['dataset'] = df['extra_data'].apply(lambda x: x.get('dataset', 'unknown'))
    df['data_source'] = df['extra_data'].apply(lambda x: x.get('data_source', 'unknown'))

    # Parse timestamps
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    df['study_parts'] = df['study_name'].str.split('_')
    df['algorithm'] = df['method']

    df['experiment_id'] = df['study_name'].apply(
        lambda x: '_'.join(x.split('_')[:-1]) if x and '_' in x else x
    )

    return df


# Load the data
print("Loading optimization results...")
results_df = load_optimization_results(DB_PATH)

if results_df.empty:
    print("No results found in the database. Please check the database path.")
else:
    print(f"Loaded {len(results_df)} optimization trials")

    # Show basic statistics
    print("\nData sources:", results_df['data_source'].unique())
    print("Datasets:", results_df['dataset'].unique())
    print("Algorithms:", results_df['algorithm'].unique())

    # Show the first few rows
    print("\nSample of the loaded data:")
    display(
        results_df[['study_name', 'trial_number', 'algorithm', 'dataset', 'data_source', 'SUR', 'DSN', 'TSN']].head(10))

Loading optimization results...
Loaded 545 optimization trials

Data sources: ['real' 'pure-synthetic' 'unknown']
Datasets: ['adult' 'synthetic_30_20' 'synthetic_2_2' 'synthetic_15_15'
 'synthetic_3_10' 'synthetic_20_3' 'synthetic_10_5' 'synthetic_10_12'
 'synthetic_10_8' 'synthetic_10_3' 'synthetic_20_5' 'synthetic_5_5'
 'synthetic_2_5' 'synthetic_50_5' 'bank' 'credit']
Algorithms: ['sg' 'adf' 'aequitas' 'expga']

Sample of the loaded data:


,study_name,trial_number,algorithm,dataset,data_source,SUR,DSN,TSN
0,real_adult_sg_1747508093,6,sg,adult,real,0.156979,1372,8740
1,real_adult_sg_1747508093,3,sg,adult,real,0.023901,318,13305
2,real_adult_sg_1747508093,2,sg,adult,real,0.236758,2512,10610
3,real_adult_sg_1747508093,1,sg,adult,real,0.091997,1161,12620
4,real_adult_sg_1747508093,0,sg,adult,real,0.056680,1134,20007
5,experiment_1747006358_config_23_adf_opt,2,adf,synthetic_30_20,pure-synthetic,0.318039,6363,20007
6,experiment_1747006358_config_23_adf_opt,1,adf,synthetic_30_20,pure-synthetic,0.335183,6706,20007
7,experiment_1747006358_config_23_adf_opt,0,adf,synthetic_30_20,pure-synthetic,0.339131,6785,20007
8,experiment_1747006358_config_23_aequitas_opt,2,aequitas,synthetic_30_20,pure-synthetic,0.182786,3657,20007
9,experiment_1747006358_config_23_aequitas_opt,1,aequitas,synthetic_30_20,pure-synthetic,0.208853,3232,15475


In [4]:
# Function to analyze best results per experiment
def analyze_best_results():
    """Extract and display the best results for each experiment"""
    if results_df.empty:
        print("No results available")
        return pd.DataFrame()

    # Group by experiment and find the best SUR value
    # Use (data_source, dataset, algorithm) as the experiment identifier
    best_results = []

    # Get unique experiments
    experiments = results_df.groupby(['data_source', 'dataset', 'algorithm'])

    for name, group in experiments:
        data_source, dataset, algorithm = name

        # Find the row with the highest SUR
        best_idx = group['SUR'].idxmax()
        best_row = group.loc[best_idx]

        # Extract relevant parameters
        best_params = best_row['parameters']

        # Keep only the most relevant parameters
        relevant_params = {}
        for param in ['threshold_rank', 'max_global', 'max_local', 'model_type',
                      'cross_rate', 'mutation', 'cluster_num', 'step_size']:
            if param in best_params:
                relevant_params[param] = best_params[param]

        best_results.append({
            'data_source': data_source,
            'dataset': dataset,
            'algorithm': algorithm,
            'best_SUR': best_row['SUR'],
            'DSN': best_row['DSN'],
            'TSN': best_row['TSN'],
            'runtime': best_row['runtime'],
            'trial_number': best_row['trial_number'],
            'relevant_params': relevant_params,
            'study_name': best_row['study_name']
        })

    # Convert to DataFrame
    best_df = pd.DataFrame(best_results)

    # Sort by best SUR descending
    best_df = best_df.sort_values(by='best_SUR', ascending=False)

    return best_df


# Analyze and display best results
best_results_df = analyze_best_results()

if not best_results_df.empty:
    print(f"Best results across {len(best_results_df)} experiments:\n")

    # Display the results without the parameters column to keep it readable
    display_cols = ['data_source', 'dataset', 'algorithm', 'best_SUR', 'DSN', 'TSN', 'runtime']
    display(best_results_df[display_cols])

    # Show details of the top 3 performing configurations
    print("\nTop 3 performing configurations:")
    for i, row in best_results_df.head(3).iterrows():
        print(f"\n{i + 1}. {row['data_source']} - {row['dataset']} - {row['algorithm']}")
        print(f"   SUR: {row['best_SUR']:.4f}, DSN: {row['DSN']}, TSN: {row['TSN']}")
        print("   Parameters:")
        for param, value in row['relevant_params'].items():
            print(f"     {param}: {value}")
else:
    print("No results to analyze")

Best results across 62 experiments:



,data_source,dataset,algorithm,best_SUR,DSN,TSN,runtime
35,pure-synthetic,synthetic_2_5,sg,0.765957,108,141,13.844282
34,pure-synthetic,synthetic_2_5,expga,0.763889,110,144,100.257659
7,pure-synthetic,synthetic_10_3,sg,0.750000,6,8,3.184629
23,pure-synthetic,synthetic_20_3,sg,0.750000,6,8,3.491005
4,pure-synthetic,synthetic_10_3,adf,0.750000,6,8,100.017163
...,...,...,...,...,...,...,...
49,real,adult,sg,0.236758,2512,10610,734.088364
38,pure-synthetic,synthetic_30_20,expga,0.216877,2529,11661,106.782654
37,pure-synthetic,synthetic_30_20,aequitas,0.208853,3232,15475,108.599256
50,unknown,adult,adf,0.186718,1746,9351,605.093658



Top 3 performing configurations:

36. pure-synthetic - synthetic_2_5 - sg
   SUR: 0.7660, DSN: 108, TSN: 141
   Parameters:
     model_type: rf
     cluster_num: 149

35. pure-synthetic - synthetic_2_5 - expga
   SUR: 0.7639, DSN: 110, TSN: 144
   Parameters:
     threshold_rank: 0.11646759543664197
     max_global: 19429
     max_local: 425
     model_type: rf
     cross_rate: 0.5955525998052242
     mutation: 0.09545624180177516

8. pure-synthetic - synthetic_10_3 - sg
   SUR: 0.7500, DSN: 6, TSN: 8
   Parameters:
     model_type: rf
     cluster_num: 149


In [7]:
from scipy import stats

conn = sqlite3.connect(DB_PATH)

query = "SELECT method, parameters, metrics FROM optimization_trials"
df = pd.read_sql_query(query, conn)
conn.close()

# Parse JSON columns
df['parameters'] = df['parameters'].apply(json.loads)
df['metrics'] = df['metrics'].apply(json.loads)
df['SUR'] = df['metrics'].apply(lambda x: x.get('SUR', 0))

# Analyze each method
methods = ['expga', 'sg', 'aequitas', 'adf']

for method in methods:
    print(f"\n===== {method.upper()} HYPERPARAMETER CORRELATION WITH SUR =====")
    
    # Filter for this method
    method_df = df[df['method'] == method].copy()
    
    if len(method_df) < 5:
        print(f"Not enough data for {method}")
        continue
    
    print(f"Number of trials: {len(method_df)}")
    
    # Get all parameters for this method
    all_params = set()
    for params in method_df['parameters']:
        all_params.update(params.keys())
    
    # Exclude non-hyperparameters
    exclude_params = ['max_runtime_seconds', 'max_tsn', 'use_cache', 'use_gpu', 'random_seed']
    params = [p for p in all_params if p not in exclude_params]
    
    # Create dataframe with parameters and SUR
    param_df = pd.DataFrame({'SUR': method_df['SUR'].values})
    
    # Extract parameters to columns
    for param in params:
        param_df[param] = method_df['parameters'].apply(lambda x: x.get(param, None))
    
    # Convert to numeric where possible
    for col in param_df.columns:
        if col != 'SUR':
            try:
                param_df[col] = pd.to_numeric(param_df[col])
            except:
                # Keep as is if not numeric
                pass
    
    # Calculate correlation and p-value for each parameter
    results = []
    
    for param in params:
        if param in param_df.columns and pd.api.types.is_numeric_dtype(param_df[param]):
            # Clean data by removing NaN values
            clean_data = param_df[['SUR', param]].dropna()
            
            if len(clean_data) > 5:  # Need enough data points
                # Calculate Pearson correlation and p-value
                corr, p_value = stats.pearsonr(clean_data[param], clean_data['SUR'])
                
                # Add to results
                results.append({
                    'parameter': param,
                    'correlation': corr,
                    'p_value': p_value,
                    'n': len(clean_data)
                })
    
    # Sort by absolute correlation
    results.sort(key=lambda x: abs(x['correlation']), reverse=True)
    
    # Print results with significance markers
    print(f"{'Parameter':<20} {'Correlation':<12} {'P-Value':<12} {'N':<6} {'Sig'}")
    print("-" * 60)
    
    for result in results:
        # Add significance markers
        sig = ''
        if result['p_value'] < 0.001:
            sig = '***'
        elif result['p_value'] < 0.01:
            sig = '**'
        elif result['p_value'] < 0.05:
            sig = '*'
        
        print(f"{result['parameter']:<20} {result['correlation']:>10.4f}  {result['p_value']:>10.4f}  {result['n']:>4}   {sig}")
    
    print("\nSignificance levels: * p<0.05, ** p<0.01, *** p<0.001")


===== EXPGA HYPERPARAMETER CORRELATION WITH SUR =====
Number of trials: 149
Parameter            Correlation  P-Value      N      Sig
------------------------------------------------------------
mutation                 0.2405      0.1299    41   
max_local               -0.2371      0.1355    41   
one_attr_at_a_time      -0.1402      0.3819    41   
max_global               0.1299      0.4182    41   
cross_rate               0.1246      0.4377    41   
threshold_rank           0.0880      0.5842    41   

Significance levels: * p<0.05, ** p<0.01, *** p<0.001

===== SG HYPERPARAMETER CORRELATION WITH SUR =====
Number of trials: 140
Parameter            Correlation  P-Value      N      Sig
------------------------------------------------------------
one_attr_at_a_time      -0.2523      0.1376    36   
cluster_num              0.2478      0.1451    36   

Significance levels: * p<0.05, ** p<0.01, *** p<0.001

===== AEQUITAS HYPERPARAMETER CORRELATION WITH SUR =====
Number of trials: 1

In [9]:
import sqlite3
import pandas as pd
import numpy as np
import json
from scipy import stats

conn = sqlite3.connect(DB_PATH)
# Get trials with dataset information
query = "SELECT method, parameters, metrics, extra_data FROM optimization_trials"
df = pd.read_sql_query(query, conn)
conn.close()

# Parse JSON columns
df['parameters'] = df['parameters'].apply(json.loads)
df['metrics'] = df['metrics'].apply(json.loads)
df['extra_data'] = df['extra_data'].apply(json.loads)
df['SUR'] = df['metrics'].apply(lambda x: x.get('SUR', 0))

# Extract dataset information
df['dataset'] = df['extra_data'].apply(lambda x: x.get('dataset', 'unknown'))
df['data_source'] = df['extra_data'].apply(lambda x: x.get('data_source', 'unknown'))

# Print basic statistics
print(f"Total trials: {len(df)}")
print(f"Methods: {', '.join(df['method'].unique())}")
print(f"Datasets: {', '.join(df['dataset'].unique())}")
print(f"Data sources: {', '.join(df['data_source'].unique())}\n")

# Function to analyze parameter consistency across datasets
def analyze_consistency(method_name):
    """Analyze how parameters correlate with SUR across different datasets"""
    # Filter for the specific method
    method_df = df[df['method'] == method_name].copy()
    
    # Get all unique parameters for this method
    all_params = set()
    for params in method_df['parameters']:
        all_params.update(params.keys())
    
    # Exclude non-hyperparameters
    exclude_params = ['max_runtime_seconds', 'max_tsn', 'use_cache', 'use_gpu', 'random_seed']
    params = [p for p in all_params if p not in exclude_params]
    
    # Get unique datasets with enough trials
    dataset_counts = method_df['dataset'].value_counts()
    valid_datasets = dataset_counts[dataset_counts >= 5].index.tolist()
    
    if len(valid_datasets) < 2:
        print(f"Not enough datasets with sufficient trials for {method_name}")
        return None
    
    # Store correlations for each dataset
    dataset_correlations = {}
    
    # Process each dataset
    for dataset in valid_datasets:
        dataset_df = method_df[method_df['dataset'] == dataset]
        
        # Store correlations for this dataset
        correlations = {}
        p_values = {}
        sample_sizes = {}
        
        # Process each parameter
        for param in params:
            # Create temporary dataframe with SUR and parameter
            param_values = []
            for _, row in dataset_df.iterrows():
                param_values.append(row['parameters'].get(param, None))
            
            temp_df = pd.DataFrame({
                'SUR': dataset_df['SUR'].values,
                'param': param_values
            })
            
            # Convert parameter to numeric if possible
            try:
                temp_df['param'] = pd.to_numeric(temp_df['param'])
                is_numeric = True
            except:
                is_numeric = False
            
            # Calculate correlation if parameter is numeric
            if is_numeric:
                # Remove NaN values
                clean_df = temp_df.dropna()
                
                if len(clean_df) >= 5:  # Need at least 5 data points
                    corr, p_val = stats.pearsonr(clean_df['param'], clean_df['SUR'])
                    
                    correlations[param] = corr
                    p_values[param] = p_val
                    sample_sizes[param] = len(clean_df)
        
        # Store results for this dataset if we have correlations
        if correlations:
            dataset_correlations[dataset] = {
                'correlations': correlations,
                'p_values': p_values,
                'sample_sizes': sample_sizes
            }
    
    if len(dataset_correlations) < 2:
        print(f"Not enough datasets with valid correlations for {method_name}")
        return None
    
    # Analyze consistency across datasets
    print(f"\n{method_name.upper()} - Analysis across {len(dataset_correlations)} datasets")
    print("-" * 80)
    
    # Get all parameters that have correlations in at least 2 datasets
    shared_params = set()
    for dataset, data in dataset_correlations.items():
        for param in data['correlations'].keys():
            if sum(1 for d in dataset_correlations.values() if param in d['correlations']) >= 2:
                shared_params.add(param)
    
    if not shared_params:
        print("No parameters shared across multiple datasets.")
        return None
    
    # Create a table of correlations across datasets
    consistency_data = []
    
    for param in shared_params:
        # Collect data for this parameter
        param_data = {
            'parameter': param,
            'datasets': 0,
            'significant_datasets': 0,
            'consistent_sign': True,
            'correlations': [],
            'p_values': [],
            'sign': None
        }
        
        # Process each dataset
        for dataset, data in dataset_correlations.items():
            if param in data['correlations']:
                corr = data['correlations'][param]
                p_val = data['p_values'][param]
                
                param_data['datasets'] += 1
                param_data['correlations'].append(corr)
                param_data['p_values'].append(p_val)
                
                # Check if significant
                if p_val < 0.05:
                    param_data['significant_datasets'] += 1
                
                # Check sign consistency
                current_sign = np.sign(corr)
                if param_data['sign'] is None:
                    param_data['sign'] = current_sign
                elif param_data['sign'] != current_sign and current_sign != 0:
                    param_data['consistent_sign'] = False
        
        # Calculate summary statistics
        param_data['mean_corr'] = np.mean(param_data['correlations'])
        param_data['std_corr'] = np.std(param_data['correlations'])
        param_data['min_corr'] = min(param_data['correlations'])
        param_data['max_corr'] = max(param_data['correlations'])
        param_data['corr_range'] = param_data['max_corr'] - param_data['min_corr']
        
        # Add to results
        consistency_data.append(param_data)
    
    # Sort by consistency and strength
    consistency_data.sort(key=lambda x: (
        x['consistent_sign'], 
        x['significant_datasets'], 
        abs(x['mean_corr'])
    ), reverse=True)
    
    # Print results
    print(f"{'Parameter':<20} {'Mean Corr':<10} {'Range':<15} {'Sign':<15} {'Significant'}")
    print("-" * 80)
    
    for data in consistency_data:
        sign_text = "Consistent" if data['consistent_sign'] else "Inconsistent"
        range_text = f"[{data['min_corr']:.3f}, {data['max_corr']:.3f}]"
        sig_text = f"{data['significant_datasets']}/{data['datasets']}"
        
        print(f"{data['parameter']:<20} {data['mean_corr']:>9.3f}  {range_text:<15} {sign_text:<15} {sig_text}")
    
    # Overall assessment
    consistent_params = [d for d in consistency_data if d['consistent_sign'] and d['significant_datasets'] > 0]
    sig_consistent_params = [d for d in consistent_params if d['significant_datasets'] >= 2]
    
    print("\nSummary:")
    print(f"- {len(consistent_params)}/{len(consistency_data)} parameters have consistent direction across datasets")
    print(f"- {len(sig_consistent_params)}/{len(consistency_data)} parameters are significant and consistent across multiple datasets")
    
    if sig_consistent_params:
        print("\nMost consistently effective parameters:")
        for i, data in enumerate(sig_consistent_params[:3]):
            direction = "positively" if data['mean_corr'] > 0 else "negatively"
            print(f"{i+1}. {data['parameter']} ({direction} correlated, significant in {data['significant_datasets']}/{data['datasets']} datasets)")
    
    return consistency_data

# Analyze each method
methods = ['expga', 'sg', 'aequitas', 'adf']

for method in methods:
    analyze_consistency(method)

Total trials: 546
Methods: sg, expga, adf, aequitas
Datasets: adult, credit, bank, synthetic_50_5, synthetic_2_5, synthetic_10_5, synthetic_5_5, synthetic_20_5, synthetic_10_3, synthetic_10_8, synthetic_10_12, synthetic_20_3, synthetic_3_10, synthetic_15_15, synthetic_2_2, synthetic_30_20
Data sources: unknown, pure-synthetic, real



C:\Users\gen06917\AppData\Local\Temp\ipykernel_7240\3329541512.py:89: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, p_val = stats.pearsonr(clean_df['param'], clean_df['SUR'])



EXPGA - Analysis across 8 datasets
--------------------------------------------------------------------------------
Parameter            Mean Corr  Range           Sign            Significant
--------------------------------------------------------------------------------
cross_rate                 nan  [-0.821, 0.865] Inconsistent    4/8
threshold_rank             nan  [-0.521, 0.884] Inconsistent    2/8
mutation                   nan  [-0.400, 0.474] Inconsistent    0/8
max_global                 nan  [-0.517, 0.436] Inconsistent    0/8
one_attr_at_a_time         nan  [nan, nan]      Inconsistent    0/8
max_local                  nan  [-0.592, 0.550] Inconsistent    0/8

Summary:
- 0/6 parameters have consistent direction across datasets
- 0/6 parameters are significant and consistent across multiple datasets

SG - Analysis across 7 datasets
--------------------------------------------------------------------------------
Parameter            Mean Corr  Range           Sign          

C:\Users\gen06917\AppData\Local\Temp\ipykernel_7240\3329541512.py:89: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, p_val = stats.pearsonr(clean_df['param'], clean_df['SUR'])
C:\Users\gen06917\AppData\Local\Temp\ipykernel_7240\3329541512.py:89: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, p_val = stats.pearsonr(clean_df['param'], clean_df['SUR'])



AEQUITAS - Analysis across 7 datasets
--------------------------------------------------------------------------------
Parameter            Mean Corr  Range           Sign            Significant
--------------------------------------------------------------------------------
step_size               -0.297  [-0.658, 0.320] Inconsistent    1/7
max_global               0.153  [-0.465, 0.739] Inconsistent    1/7
one_attr_at_a_time         nan  [nan, nan]      Inconsistent    0/7
max_local               -0.219  [-0.600, 0.032] Inconsistent    0/7
param_probability_change_size     0.179  [-0.080, 0.698] Inconsistent    0/7
init_prob               -0.171  [-0.680, 0.226] Inconsistent    0/7
direction_probability_change_size     0.035  [-0.389, 0.527] Inconsistent    0/7

Summary:
- 0/7 parameters have consistent direction across datasets
- 0/7 parameters are significant and consistent across multiple datasets

ADF - Analysis across 7 datasets
-------------------------------------------------

C:\Users\gen06917\AppData\Local\Temp\ipykernel_7240\3329541512.py:89: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, p_val = stats.pearsonr(clean_df['param'], clean_df['SUR'])
